# Библиотеки Asyncio и Aiohttp. Асинхронная загрузка данных из Интернета

In [46]:
%%capture
!pip install nest-asyncio
!pip install aiohttp
!pip install aiofiles

In [66]:
import nest_asyncio
nest_asyncio.apply()

In [67]:
import aiofiles
from  aiohttp import ClientSession
import asyncio
import os
import time
import pandas as pandas

In [76]:
async def download_data(session, url):
  start = time.time()
  filename = os.path.basename(url)
  async with session.get(url) as response:
    async with aiofiles.open(filename, 'wb') as fd:
      data = await response.content.read()
      await fd.write(data)
  print('Process {} took: {:.2f} seconds'.format(filename, time.time() - start)) 

In [77]:
async def main(loop, url):
    async with ClientSession(loop=loop) as session:
        await asyncio.wait_for(download_data(session, url), timeout=10.0)

In [78]:
urls = ["https://raw.githubusercontent.com/grishenkovp/project_cheese_market/main/project/data_sales/sales_model_0.csv",
        "https://raw.githubusercontent.com/grishenkovp/project_cheese_market/main/project/data_sales/sales_model_1.csv",
        "https://raw.githubusercontent.com/grishenkovp/project_cheese_market/main/project/data_sales/sales_model_2.csv"]

In [79]:
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(*(main(loop, url) for url in urls)))

Process sales_model_0.csv took: 0.37 seconds
Process sales_model_1.csv took: 0.37 seconds
Process sales_model_2.csv took: 0.39 seconds


[None, None, None]

### Проверка

In [86]:
df = pd.read_csv('/content/sales_model_2.csv')

In [87]:
df.head()

,date,model,retailer,customer_id,kind_cheese,quantity,price
0,2020-01-02,2,A4,81,сыр мягкий,400,578
1,2020-01-02,2,A2,93,прочие сыры,900,770
2,2020-01-02,2,A2,99,продукты сырные,950,440
3,2020-01-02,2,A2,108,продукты сырные,800,440
4,2020-01-02,2,A4,114,прочие сыры,1000,847


In [88]:
print(df.shape)

(424, 7)
